In [1]:
import requests
import pandas as pd
import json
from tqdm import tqdm

In [2]:
scenarios= 'MH Breach;IH Breach;TAS Breach;SS Breach;NL Breach;NH Breach'
user_scenarios = scenarios.split(';')
user_scenarios

['MH Breach', 'IH Breach', 'TAS Breach', 'SS Breach', 'NL Breach', 'NH Breach']

In [3]:
fed_dams = requests.get('https://fim.sec.usace.army.mil/ci/fim/getAllEAPStructure').json()
fd_df = pd.DataFrame(fed_dams)
for user_scenario in user_scenarios:
    fd_df[user_scenario] = False
fd_df

,ID,LAT,LON,NAME,LIBRARY_ID,LIBRARY_TYPE,MH Breach,IH Breach,TAS Breach,SS Breach,NL Breach,NH Breach
0,MA00964,42.118328,-71.908333,BUFFUMVILLE DAM,1117,dam_study,False,False,False,False,False,False
1,IL00005,40.671299,-89.500504,FARMDALE DAM,1113,dam_study,False,False,False,False,False,False
2,GA03742S001,34.112355,-84.713096,ALLATOONA DAM - SADDLE DIKE,1000,dam_study,False,False,False,False,False,False
3,KS00002S001,37.291890,-95.914999,ELK CITY DAM - ELK CITY LEVEE,1144,dam_study,False,False,False,False,False,False
4,CT00507S001,41.625600,-73.030588,HANCOCK BROOK DAM - RAILROAD DIKE,1444,dam_study,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
455,NM00404,35.616576,-106.317836,COCHITI DAM,7007,dam_study,False,False,False,False,False,False
456,CA10303,38.718424,-123.009620,WARM SPRINGS DAM,6807,dam_study,False,False,False,False,False,False
457,OH00031,41.299999,-80.758331,MOSQUITO CREEK DAM,6367,dam_study,False,False,False,False,False,False
458,MN00582,46.669116,-94.112818,PINE RIVER DAM,5266,dam_study,False,False,False,False,False,False


In [4]:
for dam_id in tqdm(fd_df['ID'], total=fd_df.shape[0]):
    r = requests.get("https://fim.sec.usace.army.mil/ci/fim/getEAPLayers?id=" + dam_id)
    dam_scenarios = json.loads(r.content)
    for scenario in dam_scenarios:
        for user_scenario in user_scenarios:
            if user_scenario in scenario['displayName']:
#                 print(dam_id, user_scenario)
                fd_df.loc[fd_df['ID'] == dam_id, user_scenario] = True

100%|█████████████████████████████████████████████████████████████████| 460/460 [04:38<00:00,  1.65it/s]


In [5]:
for user_scenario in user_scenarios:
    print(user_scenario, fd_df[user_scenario].sum(), round(fd_df[user_scenario].sum()/fd_df.shape[0], 2))

MH Breach 456 0.99
IH Breach 137 0.3
TAS Breach 409 0.89
SS Breach 349 0.76
NL Breach 220 0.48
NH Breach 361 0.78


In [7]:
fd_df.to_csv('./nid_available_scenario.csv')